In [1]:
import torch
from torch import nn
import numpy as np
import sys
sys.path.append("D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\code")
import d2lzh_pytorch as d2l

In [19]:
def dropout(X,drop_prob):
    assert 0<=drop_prob<=1
    keep_prob=1-drop_prob
    if keep_prob==0:
        return torch.zeros_like(X)
    ebsiron=(torch.rand(X.shape)<keep_prob).float()
    return torch.mul(X,ebsiron)/keep_prob

In [21]:
X=torch.arange(16).view(2,8)
dropout(X,0.8)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [40.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [22]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256
W1 = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs, num_hiddens1)), dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens1, requires_grad=True)
W2 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens1, num_hiddens2)), dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_hiddens2, requires_grad=True)
W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2, num_outputs)), dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs, requires_grad=True)

params = [W1, b1, W2, b2, W3, b3]

In [27]:
drop_prob1, drop_prob2 = 0.2, 0.5
def net(X,is_training=True):
    X=X.view((-1,num_inputs))
    H1=(torch.mm(X,W1)+b1).relu()
    if is_training:
        H1=dropout(H1,drop_prob1)
    H2=(torch.mm(H1,W2)+b2).relu()
    if is_training:
        H2=dropout(H2,drop_prob2)
    return torch.mm(H2,W3)+b3

In [28]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        if isinstance(net, torch.nn.Module):
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train() # 改回训练模式
        else: # 自定义的模型
            if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                # 将is_training设置成False
                acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
            else:
                acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
        n += y.shape[0]
    return acc_sum / n

In [ ]:
num_epochs, lr, batch_size = 5, 100.0, 256
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

epoch 1, loss 0.0045, train acc 0.555, test acc 0.775
epoch 2, loss 0.0024, train acc 0.777, test acc 0.784
epoch 3, loss 0.0019, train acc 0.819, test acc 0.776
epoch 4, loss 0.0018, train acc 0.838, test acc 0.840
